In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score as sil_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.base import clone

In [2]:
from sklearn.model_selection import train_test_split
from sklearn import svm
from collections import Counter
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report,confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [4]:
df = pd.read_csv('./sample-1-25.csv/sample-1-25.csv')
y_col, x_col = df.columns[0], df.columns[1::]


In [5]:
Counter(df[y_col])

Counter({0: 1987514, 1: 2490})

In [6]:
x_resampled, y_resampled = SMOTE(random_state=44).fit_resample(df[x_col], df[y_col])

In [7]:
df = pd.DataFrame(x_resampled)
df['FRD_IND'] = y_resampled
df

,APPRD_AUTHZN_CNT,AVG_DLY_AUTHZN_AMT,MRCH_CATG_CD,POS_ENTRY_MTHD_CD,RCURG_AUTHZN_IND,DISTANCE_FROM_HOME,ACCT_CURR_BAL,AUTHZN_AMT,AUTHZN_OUTSTD_AMT,PLSTC_ISU_DUR,FRD_IND
0,3,7,5812,90,0,0.000000,2498.210000,15.520000,94.280000,805,0
1,1,6,5541,90,0,0.000000,88.880000,46.560000,56.310000,68,0
2,4,0,5965,81,0,358.850300,1934.360000,16.340000,927.130000,190,0
3,0,3,5542,90,0,4.293625,916.040000,1.000000,0.000000,217,0
4,1,2,5814,90,0,509.830350,166.620000,10.660000,10.660000,110,0
...,...,...,...,...,...,...,...,...,...,...,...
3975023,2,0,8053,81,0,1997.604434,1484.799125,9.998705,34.873814,802,1
3975024,1,168,5271,90,0,4.777724,1893.551722,66.090234,83.060448,34,1
3975025,3,0,5734,81,0,2563.980640,345.248867,3.946501,21.720971,833,1
3975026,1,3,5541,90,0,21.115364,208.317144,28.344336,50.344679,679,1


In [8]:
df['Round'] = [int(np.random.rand()*10) for i in range(len(df.index))]

In [9]:
df

,APPRD_AUTHZN_CNT,AVG_DLY_AUTHZN_AMT,MRCH_CATG_CD,POS_ENTRY_MTHD_CD,RCURG_AUTHZN_IND,DISTANCE_FROM_HOME,ACCT_CURR_BAL,AUTHZN_AMT,AUTHZN_OUTSTD_AMT,PLSTC_ISU_DUR,FRD_IND,Round
0,3,7,5812,90,0,0.000000,2498.210000,15.520000,94.280000,805,0,4
1,1,6,5541,90,0,0.000000,88.880000,46.560000,56.310000,68,0,7
2,4,0,5965,81,0,358.850300,1934.360000,16.340000,927.130000,190,0,8
3,0,3,5542,90,0,4.293625,916.040000,1.000000,0.000000,217,0,0
4,1,2,5814,90,0,509.830350,166.620000,10.660000,10.660000,110,0,6
...,...,...,...,...,...,...,...,...,...,...,...,...
3975023,2,0,8053,81,0,1997.604434,1484.799125,9.998705,34.873814,802,1,1
3975024,1,168,5271,90,0,4.777724,1893.551722,66.090234,83.060448,34,1,0
3975025,3,0,5734,81,0,2563.980640,345.248867,3.946501,21.720971,833,1,5
3975026,1,3,5541,90,0,21.115364,208.317144,28.344336,50.344679,679,1,2


In [10]:
for i in range(10):
    print(np.unique(df[df['Round'] == i]['FRD_IND'], return_counts=True)[1])

[198601 198906]
[199224 198992]
[198433 198549]
[199110 198619]
[198189 199262]
[198558 198973]
[198741 198116]
[199162 199010]
[198524 198235]
[198972 198852]


## Offense

In [15]:
# MY function
def get_best_strategy_indicies(y_actual, y_pred, threshold, y_prob):  # Need to remove the y_prob since it implies that the adversary can access the classification model
    y_actual = np.array(y_actual)
    wrong_indicies = ~np.equal(y_pred,y_actual)
    fn = wrong_indicies & (y_actual & np.logical_not(y_pred))
    fn_index = np.where(fn)[0]
    
    positive, negative = confusion_matrix(y_actual, y_pred)
    assert negative[0] == len(fn_index), "Number of false negative do not match number of false negative indicies"
    return np.array(fn_index)

def append_and_smote(best_strat_indicies, x, y, smote):
    y = np.array(y)
    non_fraud_indicies = np.where(np.logical_not(y))[0]
    new_combined_indicies = np.concatenate((non_fraud_indicies, best_strat_indicies), axis=None)

    new_combined_indicies = new_combined_indicies.tolist()
    # To fix float error
    new_combined_indicies = list(map(int, new_combined_indicies))
    
    new_x = x.iloc[new_combined_indicies]
    new_y = y[new_combined_indicies]
    
    print("new_y: ", new_y)
    
    x_resampled, y_resampled = smote.fit_resample(new_x, new_y)
    
    print("y_resampled: ", y_resampled)
    
    return x_resampled, y_resampled
    

In [43]:
init_df = df[df['Round'] == 0]
x, y = init_df[init_df.columns[0:-2]], init_df[init_df.columns[-2::]]

In [45]:
y_total_data = []
ydata = []

Counter({1: 149370, 0: 148760})
Counter({0: 49841, 1: 49536})


In [46]:
# Need to run the following cell before the simulation

In [47]:
x_tr, x_te, y_tr, y_te = train_test_split(x, y['FRD_IND'], random_state=42)
print(Counter(y_tr))
print(Counter(y_te))

0.8031879460097191

## Defense

In [ ]:
# Please train the classifiers here:
# Sidd Logistic Regression
# Judd Random Forest
# Matt Decison Trees